<a href="https://colab.research.google.com/github/ssosoo/2024_DS60/blob/main/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98/%EC%84%A0%ED%98%95_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 20, 5)
    self.conv2 = nn.Conv2d(20, 20, 5)

  def foward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    return x

NameError: name 'nn' is not defined

**비선형 회귀**

non_linear.csv 파일을 사용해 비선형 회귀를 구현한다.

In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [24]:
# 사용자 정의 데이터세트
class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x = df.iloc[:, 0].values
    self.y = df.iloc[:, 1].values
    self.length = len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
    y = torch.FloatTensor([self.y[index]])
    return x,y

  def __len__(self):
    return self.length

In [29]:
# 사용자 정의 모델 선언
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__() #부모 클래스인 nn.module의 초기화 함수
    self.layer = nn.Linear(2, 1)

  def forward(self, x):
    x = self.layer(x)
    return x

In [31]:
# 사용자 정의 데이터세트와 데이터로더
train_dataset = CustomDataset("non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle=True,drop_last = True)

In [32]:
#GPU 연산 적용
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001) #매개변수 최적화

In [34]:
# 학습 진행
for epoch in range(10000):
  cost = 0.0

  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    cost+=loss

  cost = cost/len(train_dataloader)

  if (epoch + 1) % 1000 == 0:
    print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:3f}")


Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.1024, -1.7006]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.3453], device='cuda:0', requires_grad=True)], Cost : 0.074585
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.1024, -1.7031]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.3580], device='cuda:0', requires_grad=True)], Cost : 0.083377
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.1022, -1.7030]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.3695], device='cuda:0', requires_grad=True)], Cost : 0.084749
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.1020, -1.7032]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.3801], device='cuda:0', requires_grad=True)], Cost : 0.075396
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.1018, -1.7032]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.3898], device='cuda:0', requires

In [40]:
# 모델 평가
with torch.no_grad():
  model.eval()
  inputs = torch.FloatTensor(
      [
          [1**2, 1],
          [5**2, 5],
          [11**2, 11]
      ]
  ).to(device)
  outputs = model(inputs)
  print(outputs)

tensor([[  1.8251],
        [ 69.4360],
        [356.9106]], device='cuda:0')


In [41]:
# 모델 저장
# 코랩에서는 실행 불가.
torch.save(
    model,
    "../models/model.pt"
)

torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)

RuntimeError: Parent directory ../models does not exist.